In [ ]:
%%capture
!pip install langchain_openai
!pip install -U langchain-anthropic
!pip install json_repair

# Prompt Planner: A Proof of Concept for Automated Prompt Generation

The Prompt Planner is a proof of concept designed to demonstrate how large language models (LLMs) can be used to automatically generate effective prompts.

# It operates through a two-step process:

### Step 1) Information Collection:
The system begins by acting as an information collector, using LLMs to engage with users and gather necessary details based on a brief task description. This interaction is designed to build a comprehensive understanding of the task at hand.

### Step 2) Prompt Planning:
Once the detailed task description is established from the user’s inputs, this information is fed into the prompt planner. The planner then constructs a tailored prompt, optimized to instruct another LLM to produce outputs in specific formats like JSON or markdown, based on the detailed task description.


This agentic approach leverages multiple LLMs that pass information to one another, refining the process of prompt engineering to meet user expectations efficiently. The dynamically changing inputs and the ability to optimize output formats make the Prompt Planner a decent starting point for people how doesnt know how to create an effective prompts based on given inputs and desired output requirements.


In [ ]:
from langchain_openai import AzureChatOpenAI
import json_repair
from langchain_anthropic import AnthropicLLM
import os
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from bs4 import BeautifulSoup
import json
import time
from pprint import pprint

## Setup env keys for LLM, Currently only supports "openai" and "anthropic" models

In [ ]:
## Azure Open ai envs
AZURE_ENDPOINT= ""
AZURE_API_VERSION=""
AZURE_OPENAI_API_KEY= ""
AZURE_LLM_MODEL=''

## Anthropic envs
os.environ['ANTHROPIC_API_KEY']=""


## Utils funtions

In [ ]:
# Function to select and configure a language model (LLM) based on a given choice.
def get_llm(llm_choice):
    # Check if the chosen LLM is 'Anthropic'.
    if llm_choice == 'azure_openai':
        # Create an instance of AzureChatOpenAI configured for the Anthropic model.
        # This setup involves specifying various parameters like API keys and endpoint details specific to Azure.
        llm = AzureChatOpenAI(azure_endpoint=AZURE_ENDPOINT,
                              openai_api_version=AZURE_API_VERSION,
                              api_key=AZURE_OPENAI_API_KEY,
                              azure_deployment=AZURE_LLM_MODEL,
                              temperature=0, max_tokens=2000)
    # Check if the chosen LLM is 'openai'.
    elif llm_choice == 'Anthropic':
        # Create an instance of AnthropicLLM, specifically using the 'claude-2.1' model.
        llm = AnthropicLLM(model='claude-2.1')
    else:
        # If an unrecognized LLM choice is provided, raise an error.
        raise ValueError("Invalid LLM choice provided.")

    # Return the configured language model instance.
    return llm

def parse_and_save_html(content, file_name):
    """
    Parses HTML content to extract information from <section> tags and saves it to a file.

    Parameters:
    - content: A string containing the HTML content.
    - save_path: The file path where the extracted information will be stored.

    """
    soup = BeautifulSoup(content, 'html.parser')
    file_name=file_name+'.txt'
    #save_path=os.path.join(save_path, file_name)
    with open(file_name, "w") as file:
        sections = soup.find_all('section')
        for section in sections:
            # Use get to safely extract the class attribute, defaulting to an empty list
            class_name = section.get('class', [None])[0]

            # Extract the heading within each section
            heading = section.find(['h1', 'h2'])
            if heading:
                file.write(f"### {heading.text}\n")  # Write heading text with ### prefix

            # Write the content of the section
            for content in section.contents:
                if content.name not in ['h1', 'h2']:  # Avoid duplicating the heading
                    if content.name == 'ol' or content.name == 'ul':
                        # Handle each list item separately
                        list_items = content.find_all('li')
                        for item in list_items:
                            file.write(f"{item.get_text(strip=True)}\n")
                    else:
                        # Write other types of content
                        text = content.get_text(strip=True)
                        if 'output-format' == class_name:  # Check if the class is 'output-format'
                            text = text.replace('{', '{{')
                            text = text.replace('}', '}}')

                        if text:
                            file.write(f"{text}\n")
            file.write("\n")  # Add a newline for better separation between sections

    # Open the file in read mode
    with open(file_name, 'r') as file:
        content = file.read()


    print(f"#### Generated prompt is saved at {os.path.join(os.getcwd(), file_name)}")
    return content

def interact_and_generate_description(task_requ_n_desc_chain, brief_task_desc):
    """
    Interacts with the user by asking a series of questions, collects responses, and generates a task description.

    Parameters:
    - task_requ_n_desc_chain: a callable that invokes the backend model to generate questions and task descriptions.
    - initial_data: the initial input data used to start the interaction (likely the initial task description).

    Returns:
    - The final task description generated based on user responses.
    """
    chat_history = []
    user_interaction = True
    gen_ques_n_desc = task_requ_n_desc_chain.invoke({'desc': brief_task_desc, 'chat_history': chat_history})

    while user_interaction:
        # Parse the response from the chain
        response_data = json_repair.loads(gen_ques_n_desc)
        response_type = response_data['response_type']
        response = response_data['response']

        if response_type == 'questions':
            for idx, ques in enumerate(response):
                print(f"Question {idx + 1}: {ques}")
                time.sleep(2)
                user_response = input("Your answer: ")

                # Store each interaction in chat history
                chat = {'AI Message': ques, 'Human Message': user_response}
                chat_history.append(chat)

            # Notify that all questions have been answered, proceed to the next step
            print('#########################')
            print('\n\n')

            print('##  Got the user response now procedding to generate the task description so that prompt planner can plan the excecution of task##')

            print('\n\n')
            print('#########################')

            chat_history.append({'message': 'All questions answered, proceeding to generate the task description.'})
            gen_ques_n_desc = task_requ_n_desc_chain.invoke({'desc': 'create an answer based on given background context', 'chat_history': chat_history})

        elif response_type == 'task_description':
            user_interaction = False
            return response  # Return the generated task description


def runnable(llm, sys_temp, human_temp=None):
    # Convert the system template string to a SystemMessagePromptTemplate object.
    sys_temp = SystemMessagePromptTemplate.from_template(sys_temp)

    # Start with the system template in the prompts list.
    prompts_list = [sys_temp]

    # If a human template is provided, convert it to a SystemMessagePromptTemplate and append to the list.
    if human_temp:
        human_temp = SystemMessagePromptTemplate.from_template(human_temp)
        prompts_list.append(human_temp)

    # Combine all message templates into a single ChatPromptTemplate object.
    prompt = ChatPromptTemplate.from_messages(prompts_list)

    # Initialize the output parser to handle the structure of the response.
    output_parser = StrOutputParser()

    # Create a pipeline by chaining the prompt, language model (llm), and output parser together.
    # This creates a flow where the prompt is processed by the llm, and its output is parsed by output_parser.
    run = prompt | llm | output_parser

    # Return the runnable pipeline that can be executed to get the processed output.
    return run


## Prompts:

In [ ]:
info_gather_temp_sys="""
Your role is to work as a task description creator, giving a brief description of the task by the user and formulating a detailed description of the task which specifies what is the task.  The task description prepared by you will be given to the Task planner who will create a plan on the execution of the task.

Follow these two steps

Step 1) Ask Questions:
    a) Analyze the initial task description to identify any gaps in information. Formulate set of 5 questions to gather more context on the task's objective, the targeted audience, and typical scenarios where the task would be performed. The aim here is to ensure that the final task execution meets the user's expectations and satisfies their needs.
    b) Use your domain knowledge to determine what additional context is needed to ensure the task outcome aligns with the user's expectations. Consider different approaches to the task and identify areas where the user's preferences or decisions could influence execution and generate a set of questions accordingly.

Guidelines for Crafting Questions:
    1. Ensure each question is clear and direct to encourage precise answers without requiring lengthy responses.
    2. Aim for open-ended questions that elicit detailed answers but can be answered briefly and efficiently.
    3. Focus on one aspect per question to avoid overwhelming the user and keep responses focused.

Step 2) Formulate the Task Description:
    a) Once get the answers to set of questions in chat history, DO NOT generete further question.
    b) compile the information into a clear task description based on initial task description and user responses regarding their preferences and expectations.

####### Here is the chat history containing the questions and their responses ##########
chat_history:{chat_history}

###### Output Format:
    Your  output format is 'ONLY' JSON with two key responses:
    {{'response_type':"questions" or "task_description", 'response': "the list of question when the response_type is qquestions" or "the actuall task description one you get the answers of the question from the user"}}

Note:
    - Limit yourself to a maximum of five questions, Be concise and direct to avoid producing questions that overwhelm the users in responding.
    - Once you get the answers of the question you created from the chat history, formulate the task description , mask the resposne_type key to task description.
    - Output Only JSON with any text before or after
"""
info_gather_temp_human="""Heres is the brief description of the task: {desc}. Once you get the answers of step 1 questions then only proceed to step 2 and prepare the task_descriptions"""

prompt_gen_sys = """
Your role is to create a task planning prompt based on a given task description, input variables, and desired output.
The planning prompt contains step-by-step instructions on how the particular task should be executed to achieve the desired outcome.
Here the input variables are the inputs using which a task will be executed, and the desired outcome specifies how the outcome of the task should be.

Please follow the steps below to write an effective instruction prompt:

a) Thorough Understanding: Carefully read the task description to fully understand what specific task needs to be done.
b) Think and Planning: Utilize your knowledge to think about how the task needs to be done and then plan a set of step-by-step guidelines required to carry out the task effectively and accurately.
c) Guideline Review: Carefully review each guideline to ensure they logically provide clear guidance. Anyone should be able to execute the task without any prior knowledge about it.
d) Specify Output Format: Clearly mention how to construct the output format to ensure that the outcome of the task is presented as desired.

Your output for the designed task planning instruction prompt should be in the following HTML format only:
HTML format:
<html>
<body>
    <!-- Section for the Task Overview -->
    <section class="task-overview">
        <h1>Task Overview and Objective</h1>
        <p>Provide a detailed description, overview, and objective of the task here.</p>
    </section>

    <!-- Section for Step-by-Step Instructions -->
    <section class="instructions">
        <h2>Pay attention to below Step-by-Step guidelines</h2>
        <ol>
            <li>Step 1: Description of the first step.</li>
            <li>Step 2: Description of the second step.</li>
            <li>Step 3: Description of the third step.</li>
            <!-- Add more steps as needed -->
        </ol>
    </section>

    <!-- Section for Input Specification -->
    <section class="input-variables">
        <h2>Task inputs</h2>
        <p>provide the input placeholder schema in json for each variables</p> # eg if input is x then specify it in curly brackets

    </section>

    <!-- Section for Output Format Specification -->
    <section class="output-format">
        <h2>Output format Details</h2>
        <p>Specify how the outcome of the task needs to be</p>
        <p>Provide the output schema such that the output of the task is preserved. Also, specify that the outcome of the task should only be the desired output.</p>
    </section>
</body>
</html>
"""
prompt_gen_hum="""Here is the task description: {desc}, these are the input variables {variables} for the task. Desired output format: {output_format}"""



## Fill the below feilds according to your task , desired output format and llm choice

In [ ]:
# Name of the prompt, could be anything.
prompt_name = 'rag_prompt'

# A brief description of the task provided by the user, outlining what the model needs to accomplish.
task_desc_user = 'the task is to answer the query based on the given background context'

# Specifies the expected output format of the task.
# incase specifying a Json, its good to mention what each key of json will be having
exp_output_format = 'The output of the task needs to be in json with answer key and reasoning key containing the reasoning used for answering the question'

# List of variables that will be input into the LLM. Typically includes the main elements necessary for the model to perform the task.
task_input_variables = ['question', 'context']

# Specifies the provider of the LLM to be used for generating prompts. Options could be 'openai', 'Anthropic', etc.
llm_provider = 'azure_openai'  # 'Anthropic' could be another option

# Function to retrieve the specified LLM from the provider. This encapsulates the logic to interact with the chosen LLM's API.
llm = get_llm(llm_choice=llm_provider)


# Intiating the langchain chains





 This chain will be used to create a detail task description by collecting user feedback.

In [ ]:
info_collector=runnable(llm=llm, sys_temp=info_gather_temp_sys, human_temp=info_gather_temp_human )

This chain will be used to generate prompt using the information supplied to it by the previous chain.

In [ ]:
prompt_gen_chain=runnable(llm=llm, sys_temp=prompt_gen_sys, human_temp=prompt_gen_hum)


Once run the below cell, llm wil generate a set of questions and ask for an an answer one at a time

In [ ]:
gen_desc_w_hum_feedback=interact_and_generate_description(info_collector, brief_task_desc=task_desc_user)

Question 1: What is the nature of the query that needs to be answered?
Your answer: n
Question 2: Can you provide the specific background context that should be used to answer the query?
Your answer: d
Question 3: Who is the intended audience for the response to this query?
Your answer: 
Question 4: What is the preferred format for the response (e.g., written report, verbal explanation, presentation)?
Your answer: h
Question 5: Are there any specific points or aspects that must be included or emphasized in the response?
Your answer:  must be included or emphasized in the response
#########################



##Got the user response now procedding to generate the task description so that prompt planner can plan the excecution of task##



#########################
Question 1: What is the specific query or topic that needs to be addressed in the answer?
Your answer: its is realted to finance 
Question 2: What is the detailed background context that should be considered when formulating t

## Once the previous cell is executed and answers are provide to llm. Run the below cell to generate the prompt

In [ ]:
prompt_gen_response=prompt_gen_chain.invoke({'desc':gen_desc_w_hum_feedback, 'variables':str(task_input_variables),'output_format':exp_output_format })

## Run the below cell to prase and save the prompt

In [ ]:
generated_prompt=parse_and_save_html(prompt_gen_response, file_name=prompt_name)

#### Generated prompt is saved at /content/rag_prompt.txt


In [ ]:
print(generated_prompt)

### Task Overview and Objective
The task is to create a finance-related answer that is understandable by the general public. The answer should be clear and concise, avoiding the use of jargon or complex concepts that require specific background knowledge. The objective is to provide an informative response that is accessible to individuals with varying levels of expertise in finance.

### Pay attention to below Step-by-Step guidelines
Step 1: Read the provided question carefully to understand what financial information or explanation is being sought.
Step 2: Consider the context in which the question is asked to tailor your answer appropriately.
Step 3: Research the topic if necessary to ensure accuracy in your response.
Step 4: Draft a clear and concise answer, avoiding the use of technical terms or complex financial concepts that may not be widely understood.
Step 5: Provide reasoning for your answer, explaining the thought process or the basis for the information provided in a way t